# 🎓 Week 14 - Day 3: LangChain I - Building with LLMs

## Today's Goals:
✅ Understand what LangChain is and why it's useful  
✅ Work with Prompt Templates  
✅ Build and use Chains  
✅ Create conversational agents with memory  
✅ Use Runnables (modern LangChain API)

## ⏱️ Estimated Time: 90 minutes

**Note:** We'll use OpenAI models. You'll need an API key (free tier available).


## 🔧 Part 1: Setup - Install & Import All Libraries

**IMPORTANT:** Run ALL cells in this part sequentially!


In [1]:
# STEP 1: Install required packages

!pip install -q langchain==0.1.20
!pip install -q langchain-openai==0.1.7
!pip install -q langchain-community==0.0.38

print("✅ LangChain libraries installed!")


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
sip 6.12.0 requires packaging>=24.2, but you have packaging 23.2 which is incompatible.


✅ LangChain libraries installed!


In [2]:
# STEP 2: Import libraries

from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.memory import ConversationBufferMemory
import os

print("✅ Libraries imported!")


✅ Libraries imported!


In [3]:
# STEP 3: Setup API key

# Set your OpenAI API key
# Get free key: https://platform.openai.com/api-keys
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

print("✅ API configured!")
print("💡 Replace 'your-api-key-here' with your actual API key!")


✅ API configured!
💡 Replace 'your-api-key-here' with your actual API key!


### 💡 Key Insights:
- **LangChain** = Framework for LLM applications
- **langchain-openai** for OpenAI models
- Need **API key** to use OpenAI models


## 🤖 Part 2: Your First LangChain LLM Call

Start simple - make a direct LLM call.


In [4]:
# STEP 1: Initialize the LLM

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7
)

print("✅ LLM initialized!")
print(f"Model: {llm.model_name}")


✅ LLM initialized!
Model: gpt-3.5-turbo


In [5]:
# STEP 2: Make a simple call

response = llm.invoke("Explain LangChain in one sentence")

print("🤖 Response:")
print(response.content)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-api*****here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### 💡 Key Insights:
- **ChatOpenAI** wraps OpenAI API
- **invoke()** sends prompt, gets response
- **temperature** controls creativity (0-1)


## 📝 Part 3: Prompt Templates - Reusable Prompts

Templates make prompts reusable with variables.


In [ ]:
# STEP 1: Create a prompt template

template = "Tell me a {adjective} joke about {topic}"

prompt = PromptTemplate(
    input_variables=["adjective", "topic"],
    template=template
)

print("✅ Prompt template created!")
print(f"Template: {template}")


In [ ]:
# STEP 2: Use template with different inputs

# Example 1
p1 = prompt.format(adjective="funny", topic="programming")
print("Example 1:", p1)

# Example 2
p2 = prompt.format(adjective="short", topic="AI")
print("Example 2:", p2)


In [ ]:
# STEP 3: Combine template with LLM

formatted = prompt.format(adjective="clever", topic="Python")
response = llm.invoke(formatted)

print("🤖 Joke:")
print(response.content)


### 💡 Key Insights:
- **Templates** = Reusable prompts
- **Variables** in {braces}
- **Cleaner code** - separate logic from data


## ⛓️ Part 4: Chains - Connecting Components

Chains combine prompts + LLMs into workflows.


In [ ]:
# STEP 1: Create an LLMChain

template = "You are a helpful assistant. Answer: {question}"

prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)

print("✅ LLMChain created!")


In [ ]:
# STEP 2: Run the chain

response = chain.invoke({"question": "What is LangChain?"})

print("🤖 Response:")
print(response["text"])


In [ ]:
# STEP 3: Use chain multiple times

questions = [
    "What is a prompt template?",
    "What are chains?",
    "What is an agent?"
]

for q in questions:
    response = chain.invoke({"question": q})
    print(f"Q: {q}")
    print(f"A: {response['text'][:80]}...")
    print()


### 💡 Key Insights:
- **LLMChain** = Prompt + LLM
- **Reusable** workflow
- Call with different inputs


## 🔗 Part 5: Sequential Chains - Multi-Step Workflows

Chain multiple LLM calls together!


In [ ]:
# STEP 1: Create first chain

prompt1 = PromptTemplate(
    input_variables=["topic"],
    template="Generate a story idea about {topic}. 1-2 sentences."
)

chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="idea")

print("✅ Chain 1: Story idea")


In [ ]:
# STEP 2: Create second chain

prompt2 = PromptTemplate(
    input_variables=["idea"],
    template="Expand this idea: {idea}. Give 3 plot points."
)

chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="outline")

print("✅ Chain 2: Outline")


In [ ]:
# STEP 3: Combine into sequential chain

from langchain.chains import SimpleSequentialChain

full_chain = SimpleSequentialChain(
    chains=[chain1, chain2],
    verbose=True
)

print("✅ Sequential chain!")
print("Flow: Topic → Idea → Outline")


In [ ]:
# STEP 4: Run sequential chain

result = full_chain.invoke("space exploration")

print("📝 Final Outline:")
print(result["output"])


### 💡 Key Insights:
- **Sequential** = Multiple steps
- Output1 → Input2
- **Multi-step** tasks


## 💬 Part 6: Conversational Chains - Chatbots with Memory

Build chatbot that remembers conversation!


In [ ]:
# STEP 1: Create memory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

print("✅ Memory initialized!")


In [ ]:
# STEP 2: Create conversational chain

from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

print("✅ Conversational chain!")


In [ ]:
# STEP 3: Have a conversation

# Message 1
r1 = conversation.predict(input="Hi! I'm Alex, I love Python.")
print(f"Human: Hi! I'm Alex, I love Python.")
print(f"AI: {r1[:100]}...")
print()

# Message 2
r2 = conversation.predict(input="What's my name?")
print(f"Human: What's my name?")
print(f"AI: {r2}")
print()

# Message 3
r3 = conversation.predict(input="What language did I mention?")
print(f"Human: What language did I mention?")
print(f"AI: {r3}")


### 💡 Key Insights:
- **Memory** stores history
- Chain **remembers** context
- Perfect for **chatbots**


## 🏃 Part 7: Runnables - Modern LangChain API

The new, cleaner way to work!


In [ ]:
# STEP 1: Create chain with LCEL

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Tell me a fact about {topic}"
)

# Modern chain with pipe
chain = prompt | llm | StrOutputParser()

print("✅ Runnable chain!")


In [ ]:
# STEP 2: Use invoke()

result = chain.invoke({"topic": "AI"})

print("🤖 Result:")
print(result)


In [ ]:
# STEP 3: Use batch()

topics = [
    {"topic": "ML"},
    {"topic": "DL"},
    {"topic": "NLP"}
]

results = chain.batch(topics)

for i, r in enumerate(results, 1):
    print(f"{i}. {r}")


### 💡 Key Insights:
- **Runnables** = Modern API
- **Pipe |** chains components
- **Cleaner, Pythonic**


## 🤖 Part 8: Simple Conversational Agent

Create smart agent with context!


In [ ]:
# STEP 1: Create agent with system prompt

from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

chain = prompt | llm

print("✅ Agent chain created!")


In [ ]:
# STEP 2: Create conversation loop

chat_history = []

def chat(message):
    response = chain.invoke({
        "chat_history": chat_history,
        "input": message
    })
    
    # Add to history
    chat_history.append(HumanMessage(content=message))
    chat_history.append(response)
    
    return response.content

print("✅ Chat function ready!")


In [ ]:
# STEP 3: Test the agent

print("💬 Conversation:\n")

# Turn 1
r1 = chat("Hi! I'm learning about LangChain.")
print(f"You: Hi! I'm learning about LangChain.")
print(f"AI: {r1}\n")

# Turn 2
r2 = chat("What was I just talking about?")
print(f"You: What was I just talking about?")
print(f"AI: {r2}\n")

# Turn 3
r3 = chat("Give me 3 tips for learning it.")
print(f"You: Give me 3 tips for learning it.")
print(f"AI: {r3}")


### 💡 Key Insights:
- **System prompts** set behavior
- **MessagesPlaceholder** for history
- **Manual memory** management


## 🎯 Challenge Time!

### 🏆 Beginner Challenge:

**Task:** Build a translator chatbot!

**Requirements:**
1. User can ask to translate sentences
2. Bot remembers what language they're translating to
3. Use ConversationChain with memory

**Example:**
```
User: "I want to translate to Spanish"
Bot: "Great! I'll translate to Spanish for you."

User: "Hello, how are you?"
Bot: "Hola, ¿cómo estás?"
```

**Hint:** Use the ConversationChain from Part 6!

**Try it yourself!** 🚀


## 📚 Summary: What We Learned

### ✅ Key Concepts:

**1. LangChain:**
- Framework for LLM applications
- Pre-built components
- Production-ready patterns

**2. Prompt Templates:**
- Reusable prompts with variables
- Clean separation of logic and data
- Easy to maintain

**3. Chains:**
- Connect components into workflows
- LLMChain = basic building block
- Sequential chains for multi-step

**4. Memory:**
- Store conversation history
- Enable contextual responses
- Perfect for chatbots

**5. Runnables:**
- Modern LangChain API
- Pipe operator |
- invoke(), batch(), stream()

**6. Agents:**
- LLMs that make decisions
- Use tools and maintain context
- Next level of capability

### 🎯 Key Takeaways:

1️⃣ LangChain simplifies LLM app development

2️⃣ Templates make prompts reusable

3️⃣ Chains connect components into workflows

4️⃣ Memory enables natural conversations

5️⃣ Runnables provide clean, modern API

---

## 🚀 Next Steps:

- **Day 4:** LangChain II + LangSmith + LangGraph
- **Practice:** Build your own chatbot
- **Explore:** LangChain documentation
- **Experiment:** Try different chain combinations

---

## 📖 Resources:

- **LangChain Docs:** https://python.langchain.com/docs/
- **LangChain GitHub:** https://github.com/langchain-ai/langchain
- **Tutorials:** https://python.langchain.com/docs/tutorials/

---

**🎉 Congratulations! You've learned LangChain basics! 🎉**
